# Exploratory Data Analysis for Australia and New Zealand Banking Group


## **Introduction**


This project aims to garner insights from transaction data, through data manipulation, descriptive statistics and visual exploration. The dataset contained in this project was provided by ***Australia and New Zealand Banking Group*** for a virtual internship program they offer. The data is a synthensized representation of transactions over a three month period. I hope you enjoy following this analysis as much as i enjoyed putting it together.



#### **Visual inspection and Cleaning**

To begin the exploration, we should start by loading all the required libraries. The package "tidyverse" contains a host of libraries with functionalities such as data visualization with ***ggplot2***, data manipulation with ***dplyr***,reading excel files with ***readxl*** and working with dates and time with ***lubridate*** that will be useful for our journey. Next we read-in the data and observe its structure. We should also check the first and last few values to get a feel for the dataset.

In [ ]:
#load required library
library(tidyverse)
library(readxl)
library(lubridate)

In [ ]:
#read file as "ANZ"
anz_xlsx <- "../input/anz-transaction/anz.xlsx"
ANZ <- read_excel(anz_xlsx)

In [ ]:
#observe file structure
glimpse(ANZ)

In [ ]:
#print first 10 values
head(ANZ, 10)

In [ ]:
#COUNT ROW NUMBERS 12043 rows
nrow(ANZ)

In [ ]:
#check for repeated transactions#
ANZ%>%  group_by(transaction_id) %>%
  count(sort = TRUE) %>% 
  filter(n > 1)

The code chunks above reveal that they are 12043 observations(rows) and 23 variables(columns). All 12043 are unique transactions as the code cell above show. The columns *bpay_biller_code* and *merchant_code* seem to be full of null values.A small investigation has revealed that **"bpay"** is a payment system, it makes sense that most transactions didn't come through this system. However, futher inspection showed that all entries in those rows are either missing or have the value 0, as such they will be filtered out for the remainder of this analysis.  

In [ ]:
#create new dataset without rows with missing values called "ANZ_sparkly"
ANZ_sparkly <- ANZ %>% 
                 select(-merchant_code, -bpay_biller_code) 

#print new data set
ANZ_sparkly

Turns out they are still some missing values. However, these values only seem to be missing from "posted" transactions. If this is the case it might be that ANZ no longer needs store merchant information after transactions have been confirmed(posted). To confirm whether all missing values are from "authorised" transactions run the code cell below.

In [ ]:
#merchant id and card_present, merchant_surburb, merchant_state, merchant_long_lat are missing for posted transactions.
ANZ_sparkly %>% 
#group data by status column
 group_by(status) %>% 
#count all missing values per status in the listed columns
 summarize(sum(is.na(card_present_flag)),sum(is.na(merchant_id)), sum(is.na(merchant_suburb)), sum(is.na(merchant_long_lat)))

Before moving forward we should also check if all values are in the same currency we can do this with the help of the R code below.

In [ ]:
#confirm all transactions are the same currency
  ANZ_sparkly %>%
  group_by(currency) %>%
  count()

4326 transactions are "posted" and so all merchant information in these transactions are missing, We should only exclude them from any analysis requiring merchant information. This concludes the cleaning portion of our analysis, next we describe the data with some statistics.

### Descriptive Statistics

The following code cells contain basic statistics concerning transaction volume

In [ ]:
#statistics of transaction volume
summary(ANZ_sparkly$amount)

The code block below aggregates the data by movement(credit/debit) and calculates the percentage of the total transaction volume each channel is responsible for. It shows that about 74% of all transactions are credit transactions, while about 26% are credit transactions.

In [ ]:
#11160 transactions where debits while 883 are credit transactions.
ANZ_sparkly %>% 
  group_by(movement) %>% 
  summarize(amount_per_movement = sum(amount), no_of_transactions = n()) %>%
  mutate(percentage_movement = amount_per_movement/sum(amount_per_movement) * 100)


### Timeseries Analysis

To analysis the data by time we need to extract information based on the *date* and *extraction* columns in the dataset. The package "lubridate" contains functions specially designed for this purpose. The next few code cells below demonstrate this.

In [ ]:
#extract datetime information and into new columns in the dataset
ANZ_sparkly <- ANZ_sparkly %>%
                   #mutate() adds new columns and the time related functions help extract information
                   mutate(datetime = ymd_hms(extraction)) %>%
                   mutate(
                     yday = yday(datetime),
                     hour = hour(datetime),
                     minute = minute(datetime),
                     second = second(datetime),
                     time_of_day = ifelse(am(datetime), "AM", "PM"),
                     weekday = wday(datetime, label = TRUE),
                     month = month(datetime, label = TRUE),
                     date = ymd(date)
                          )
#see new columns
head(ANZ_sparkly %>% select(datetime,yday, hour, weekday,time_of_day, month, date))

In [ ]:
#create a more concise data frame for analysis
#select() helps to choose relevant columns
ANZ_sparkly_analysis <- ANZ_sparkly %>% 
                            select(
                              status,
                              account,
                              balance,
                              txn_description, 
                              date, 
                              age,
                              amount,
                              country,
                              gender,
                              movement,
                              datetime, 
                              hour,
                              yday,
                              minute,
                              second, 
                              time_of_day, 
                              weekday, 
                              month,
                              date)

Now that we have columns with neatly extracted and formated date objects, we can now perform some basic time series analysis. The following code cells are aimed at finding insights such as:

1. The transaction volume per day.
2. The daily mean and median
3. The transaction volume per month
4. the monthly average, maximum, median etc.
5. The transaction volume per week and other weekly based statistics.

In [ ]:
#transaction volume by day as "anz_daily"
anz_daily <- ANZ_sparkly %>% 
#group_by(date) instructs R to aggregate data by date in this case
              group_by(date) %>%
              summarize(
               daily_total = sum(amount),
               daily_mean = mean(amount),
               daily_median = median(amount),
               daily_max = max(amount),
               daily_min = min(amount),
               daily_sd = sd(amount)) %>% 
 mutate(average_daily_transaction = sum(daily_total)/length(date), 
        average_daily_max = mean(daily_max),
        average_daily_min = mean(daily_min),
        average_daily_sd = mean(daily_sd), 
        average_daily_mean = mean(daily_mean)) 
#print "anz_daily"
anz_daily

The table above shows the total transaction volume, maximum transaction and more for each day in the data frame. It also shows the average daily transactions over time. To understand these daily transactions more, we can visualize them.


In [ ]:
#plot daily amount over time, faceted by movement
ggplot(anz_daily, aes(date, daily_total)) + 
  geom_line(col= "darkblue") + 
   scale_x_date(date_breaks = "7 days" , date_labels = "%Y-%m-%d") +
    theme(axis.text.x = element_text(angle = 65, hjust = 1)) 
     

R can help us classify the data by movement type before seeing daily trends

In [ ]:
#group_by(date) instructs R to aggregate data by date in this case
ANZ_sparkly %>% 
  group_by(movement,date) %>%
  summarize(daily_total = sum(amount)) %>%
  ggplot(aes(date, daily_total)) + 
   geom_line(col = "darkred") +
   scale_x_date(date_breaks = "10 days" , date_labels = "%A") +
   theme(axis.text.x = element_text(angle = 65, hjust = 1)) + 
   facet_wrap(~movement)
   

   

It seems traffic levels differ with weekdays, to confirm this we can visualize transaction volume by weekday with the help of a barplot.

In [ ]:
#plot amount by weekday and faceted by movement 
ANZ_sparkly %>% 
  group_by(weekday) %>%
  mutate(amount_by_week = sum(amount)) %>%
  ungroup()%>%
  ggplot(aes(weekday,amount_by_week, fill = time_of_day)) + geom_col() +
  facet_wrap(~movement)

The plot above revealed that they were no credit transactions on saturdays and sundays, perhaps due to policy or technological constraints. The plot also revealed that the highest volume of transactions occur on friday evenings.The code cells below show the exact figures


In [ ]:
ANZ_sparkly %>% 
  group_by(weekday,time_of_day) %>%
  summarise(total = sum(amount), 
            mean = mean(amount),
            median = median(amount),
            max = max(amount), 
            min = min(amount),
            standard_deviation = sd(amount)) %>%
  ungroup()%>%
  mutate(percentage_of_transactions = total/sum(total)*100)

Finally, we can repeat the previous analysis and visualizations but by month.

In [ ]:
#transaction volume by month
ANZ_sparkly %>% 
  group_by(month) %>%
  summarise(total_per_month = sum(amount),
            mean_per_month = mean(amount),
            median_per_month = median(amount),
            max_per_month = max(amount),
            min_per_month = min(amount),
            standard_deviation = sd(amount)) %>%
  mutate(monthly_average = sum(total_per_month)/3)
  

In [ ]:
#transaction volume by month
ANZ_sparkly %>% 
  group_by(month) %>%
  mutate(total_per_month = sum(amount)) %>%
  ungroup() %>%
  ggplot(aes(month,total_per_month, fill = time_of_day)) + geom_col() +
  facet_wrap(~movement)
  

## Conclusion and recommendations

The analysis has revealed some valuable insights about the data, including:
1. The average daily transaction
2. the total transaction volume per month
3. the weekday and time periods with the most transaction volumes.